In [127]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re

In [131]:
movies_pd = pd.read_csv('./data2.csv', encoding='utf-8')
movies_pd.columns = ['movie_id', 'title', 'star', 'movie_rating', 'genre', 'director', 'actors', 'summary']

In [ ]:
# 전처리
for count,i in enumerate(movies_pd['genre']):
  if i is not np.nan:
      text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', i)
      movies_pd['genre'][count] = text
  else:
      n = " "     # nan -> " "
      movies_pd['genre'][count] = n

for count, i in enumerate(movies_pd['actors']):
  if i is not np.nan:
      text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', i)
      movies_pd['actors'][count] = text
  else:
      i = " "     # nan -> " "
      movies_pd['actors'][count] = n

In [133]:
# 결측치 처리
# movie_id, title, star, movie_rating, director, summary

movies_pd['movie_id'] = movies_pd['movie_id'].replace(np.nan, " ")
movies_pd['title'] = movies_pd['title'].replace(np.nan, " ")
movies_pd['star'] = movies_pd['star'].replace(np.nan, " ")
movies_pd['movie_rating'] = movies_pd['movie_rating'].replace(np.nan, " ")
movies_pd['director'] = movies_pd['director'].replace(np.nan, " ")
movies_pd['summary'] = movies_pd['summary'].replace(np.nan, " ")
movies_pd['summary'] = movies_pd['summary'].str.replace('[^\w]', '')

In [94]:
# !pip install nltk
# !pip install konlpy
# !pip install kss

In [135]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [136]:
# summary tokenize
from konlpy.tag import Okt, Kkma
from collections import Counter
from nltk.corpus import stopwords

sentence = movies_pd['summary']
for text in sentence:
  words = Kkma().morphs(text)

  # stopword (한글+영어 -> lower())
  hangul_sw = ['{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'','은', '는', '이', '가', ',', '.', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로', '바꾸어서 말하면', '바꾸어서 한다면', '만약', '그렇지않으면', '까악', '툭', '딱', '삐걱거리다', '보드득', '비걱거리다', '꽈당', '응당', '해야한다', '에 가서', '각', '각각', '여러분', '각종', '각자', '제각기', '하도록하다', '와', '과', '그러므로', '그래서', '고로', '한 까닭에', '하기 때문에', '거니와', '이지만', '대하여', '관하여', '관한', '과연', '실로', '아니나다를가', '생각한대로', '진짜로', '한적이있다', '하곤하였다', '하', '하하', '허허', '아하', '거바', '와', '오', '왜', '어째서', '무엇때문에', '어찌', '하겠는가', '무슨', '어디', '어느곳', '더군다나', '하물며', '더욱이는', '어느때', '언제', '야', '이봐', '어이', '여보시오', '흐흐', '흥', '휴', '헉헉', '헐떡헐떡', '영차', '여차', '어기여차', '끙끙', '아야', '앗', '아야', '콸콸', '졸졸', '좍좍', '뚝뚝', '주룩주룩', '솨', '우르르', '그래도', '또', '그리고', '바꾸어말하면', '바꾸어말하자면', '혹은', '혹시', '답다', '및', '그에 따르는', '때가 되어', '즉', '지든지', '설령', '가령', '하더라도', '할지라도', '일지라도', '지든지', '몇', '거의', '하마터면', '인젠', '이젠', '된바에야', '된이상', '만큼\t어찌됏든', '그위에', '게다가', '점에서 보아', '비추어 보아', '고려하면', '하게될것이다', '일것이다', '비교적', '좀', '보다더', '비하면', '시키다', '하게하다', '할만하다', '의해서', '연이서', '이어서', '잇따라', '뒤따라', '뒤이어', '결국', '의지하여', '기대여', '통하여', '자마자', '더욱더', '불구하고', '얼마든지', '마음대로', '주저하지 않고', '곧', '즉시', '바로', '당장', '하자마자', '밖에 안된다', '하면된다', '그래', '그렇지', '요컨대', '다시 말하자면', '바꿔 말하면', '즉', '구체적으로', '말하자면', '시작하여', '시초에', '이상', '허', '헉', '허걱', '바와같이', '해도좋다', '해도된다', '게다가', '더구나', '하물며', '와르르', '팍', '퍽', '펄렁', '동안', '이래', '하고있었다', '이었다', '에서', '로부터', '까지', '예하면', '했어요', '해요', '함께', '같이', '더불어', '마저', '마저도', '양자', '모두', '습니다', '가까스로', '하려고하다', '즈음하여', '다른', '다른 방면으로', '해봐요', '습니까', '했어요', '말할것도 없고', '무릎쓰고', '개의치않고', '하는것만 못하다', '하는것이 낫다', '매', '매번', '들', '모', '어느것', '어느', '로써', '갖고말하자면', '어디', '어느쪽', '어느것', '어느해', '어느 년도', '라 해도', '언젠가', '어떤것', '어느것', '저기', '저쪽', '저것', '그때', '그럼', '그러면', '요만한걸', '그래', '그때', '저것만큼', '그저', '이르기까지', '할 줄 안다', '할 힘이 있다', '너', '너희', '당신', '어찌', '설마', '차라리', '할지언정', '할지라도', '할망정', '할지언정', '구토하다', '게우다', '토하다', '메쓰겁다', '옆사람', '퉤', '쳇', '의거하여', '근거하여', '의해', '따라', '힘입어', '그', '다음', '버금', '두번째로', '기타', '첫번째로', '나머지는', '그중에서', '견지에서', '형식으로 쓰여', '입장에서', '위해서', '단지', '의해되다', '하도록시키다', '뿐만아니라', '반대로', '전후', '전자', '앞의것', '잠시', '잠깐', '하면서', '그렇지만', '다음에', '그러한즉', '그런즉', '남들', '아무거나', '어찌하든지', '같다', '비슷하다', '예컨대', '이럴정도로', '어떻게', '만약', '만일', '위에서 서술한바와같이', '인 듯하다', '하지 않는다면', '만약에', '무엇', '무슨', '어느', '어떤', '아래윗', '조차', '한데', '그럼에도 불구하고', '여전히', '심지어', '까지도', '조차도', '하지 않도록', '않기 위하여', '때', '시각', '무렵', '시간', '동안', '어때', '어떠한', '하여금', '네', '예', '우선', '누구', '누가 알겠는가', '아무도', '줄은모른다', '줄은 몰랏다', '하는 김에', '겸사겸사', '하는바', '그런 까닭에', '한 이유는', '그러니', '그러니까', '때문에', '그', '너희', '그들', '너희들', '타인', '것', '것들', '너', '위하여', '공동으로', '동시에', '하기 위하여', '어찌하여', '무엇때문에', '붕붕', '윙윙', '나', '우리', '엉엉', '휘익', '윙윙', '오호', '아하', '어쨋든', '만 못하다\t하기보다는', '차라리', '하는 편이 낫다', '흐흐', '놀라다', '상대적으로 말하자면', '마치', '아니라면', '쉿', '그렇지 않으면', '그렇지 않다면', '안 그러면', '아니었다면', '하든지', '아니면', '이라면', '좋아', '알았어', '하는것도', '그만이다', '어쩔수 없다', '하나', '일', '일반적으로', '일단', '한켠으로는', '오자마자', '이렇게되면', '이와같다면', '전부', '한마디', '한항목', '근거로', '하기에', '아울러', '하지 않도록', '않기 위해서', '이르기까지', '이 되다', '로 인하여', '까닭으로', '이유만으로', '이로 인하여', '그래서', '이 때문에', '그러므로', '그런 까닭에', '알 수 있다', '결론을 낼 수 있다', '으로 인하여', '있다', '어떤것', '관계가 있다', '관련이 있다', '연관되다', '어떤것들', '에 대해', '이리하여', '그리하여', '여부', '하기보다는', '하느니', '하면 할수록', '운운', '이러이러하다', '하구나', '하도다', '다시말하면', '다음으로', '에 있다', '에 달려 있다', '우리', '우리들', '오히려', '하기는한데', '어떻게', '어떻해', '어찌됏어', '어때', '어째서', '본대로', '자', '이', '이쪽', '여기', '이것', '이번', '이렇게말하자면', '이런', '이러한', '이와 같은', '요만큼', '요만한 것', '얼마 안 되는 것', '이만큼', '이 정도의', '이렇게 많은 것', '이와 같다', '이때', '이렇구나', '것과 같이', '끼익', '삐걱', '따위', '와 같은 사람들', '부류의 사람들', '왜냐하면', '중의하나', '오직', '오로지', '에 한하다', '하기만 하면', '도착하다', '까지 미치다', '도달하다', '정도에 이르다', '할 지경이다', '결과에 이르다', '관해서는', '여러분', '하고 있다', '한 후', '혼자', '자기', '자기집', '자신', '우에 종합한것과같이', '총적으로 보면', '총적으로 말하면', '총적으로', '대로 하다', '으로서', '참', '그만이다', '할 따름이다', '쿵', '탕탕', '쾅쾅', '둥둥', '봐', '봐라', '아이야', '아니', '와아', '응', '아이', '참나', '년', '월', '일', '령', '영', '일', '이', '삼', '사', '오', '육', '륙', '칠', '팔', '구', '이천육', '이천칠', '이천팔', '이천구', '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '령', '영']
  english_sw = stopwords.words('english')
  sw = hangul_sw + english_sw

  nsw_words = []
  for x in words:
    if x not in sw:
      nsw_words.append(x.lower())

KeyboardInterrupt: ignored

In [137]:
# TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range=(1, 2) 는 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(movies_pd['genre'])
print(tfidf_vec.vocabulary_.items())
# 4803은 영화의 개수, 276은 단어의 개수 -> 하나의 영화를 276개 열을 가진 벡터로 표현
print(tfidf_matrix.shape)

dict_items([('코미디', 304), ('드라마', 68), ('멜로로맨스', 96), ('코미디 드라마', 308), ('드라마 멜로로맨스', 73), ('범죄', 157), ('액션', 235), ('스릴러', 192), ('범죄 액션', 169), ('액션 드라마', 240), ('드라마 코미디', 86), ('코미디 스릴러', 315), ('전쟁', 278), ('전쟁 드라마', 283), ('이탈리아', 270), ('한국', 356), ('대만', 58), ('가족', 15), ('코미디 액션', 317), ('액션 가족', 237), ('홍콩', 360), ('sf', 0), ('판타지', 326), ('sf 판타지', 14), ('판타지 코미디', 340), ('모험', 111), ('액션 모험', 242), ('모험 코미디', 125), ('코미디 판타지', 320), ('중국', 292), ('드라마 범죄', 77), ('범죄 멜로로맨스', 162), ('멜로로맨스 코미디', 109), ('액션 코미디', 250), ('공포', 30), ('공포 스릴러', 38), ('멜로로맨스 드라마', 99), ('애니메이션', 222), ('코미디 가족', 306), ('가족 판타지', 29), ('판타지 범죄', 335), ('범죄 애니메이션', 168), ('범죄 드라마', 161), ('드라마 액션', 83), ('액션 sf', 236), ('sf 드라마', 3), ('sf 스릴러', 9), ('스릴러 액션', 201), ('미스터리', 147), ('액션 공포', 238), ('공포 미스터리', 36), ('미스터리 sf', 148), ('국가불명', 43), ('프랑스', 342), ('액션 범죄', 245), ('코미디 범죄', 313), ('sf 모험', 5), ('가족 액션', 26), ('코미디 멜로로맨스', 309), ('범죄 sf', 158), ('sf 가족', 1), ('가족 모험', 20), ('모험 애니메이션', 122

In [139]:
from sklearn.metrics.pairwise import cosine_similarity

# 유사도 (자기자신=1)
genres_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
# 유사도 값이 높은 영화의 제목, 인덱스 뽑기
similar_index = np.argsort(-genres_similarity)
print(similar_index)

In [140]:
input_movie = input()

movie_index = movies_pd[movies_pd['title']==input_movie].index.values
print(movie_index)
similar_movies = similar_index[movie_index, :10]

# 인덱스로 사용하기 위해서 1차원으로 변형
similar_movies_index = similar_movies.reshape(-1)

print(movies_pd.iloc[similar_movies_index]['title'])
print(movies_pd.iloc[similar_movies_index]['star'])

원더
[]
Series([], Name: title, dtype: object)
Series([], Name: star, dtype: float64)


In [ ]:
# Count(빈도수)  시간이 오조조조오오오오오오오억걸림.............!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# count_vec = CountVectorizer(ngram_range=(1, 2))
# count_matrix = count_vec.fit_transform(movies_pd['genre'])

# genres_similarity2 = cosine_similarity(count_matrix, count_matrix)
# similar_index = np.argsort(-genres_similarity2)

# input_movie = input()
# movie_index = movies_pd[movies_pd['title']==input_movie].index.values
# print(movie_index)
# similar_movies = similar_index[movie_index, :10]

# # 인덱스로 활용하기 위해 1차원변형
# similar_movies_index = similar_movies.reshape(-1)
# print(movies_pd.iloc[similar_movies_index]['title'] )